In [1]:
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
import pyspark.sql.types as t
import pyspark.sql.functions as f
import seaborn as sns
import matplotlib.dates as mdates
import matplotlib.pyplot as plt

In [10]:
spark = (SparkSession.builder.config("spark.driver.memory","5g").config("spark.driver.maxResultSize", "5g").getOrCreate())
spark

In [3]:
%%time
from pyspark.sql.types import DoubleType


X_train = spark.read.csv("./Dataset/train.csv", header=True, inferSchema=True)
X_test = spark.read.csv("./Dataset/example_test.csv", header=True, inferSchema=True)

# Convert VWAP Feature to double
X_train = X_train.withColumn("VWAP", X_train.VWAP.cast(DoubleType()))

X_train.printSchema()

root
 |-- timestamp: integer (nullable = true)
 |-- Asset_ID: integer (nullable = true)
 |-- Count: double (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Volume: double (nullable = true)
 |-- VWAP: double (nullable = true)
 |-- Target: double (nullable = true)

CPU times: user 10.1 ms, sys: 2.17 ms, total: 12.3 ms
Wall time: 31.2 s


In [4]:
X_train.head(1)

[Row(timestamp=1514764860, Asset_ID=2, Count=40.0, Open=2376.58, High=2399.5, Low=2357.14, Close=2374.59, Volume=19.23300519, VWAP=2373.1163915061647, Target=-0.004218152387429286)]

In [13]:
%%time
# Check null values if any

X_train.select([f.count(f.when(f.isnull(c), c)).alias(c) for c in X_train.columns]).show()

+---------+--------+-----+----+----+---+-----+------+----+------+
|timestamp|Asset_ID|Count|Open|High|Low|Close|Volume|VWAP|Target|
+---------+--------+-----+----+----+---+-----+------+----+------+
|        0|       0|    0|   0|   0|  0|    0|     0|   9|750338|
+---------+--------+-----+----+----+---+-----+------+----+------+

CPU times: user 12.2 ms, sys: 8.13 ms, total: 20.3 ms
Wall time: 22.5 s


In [14]:
# Fill null values with mean
from pyspark.ml.feature import Imputer
imputer = Imputer(
    inputCols = ['VWAP', 'Target'],
    outputCols = ["{}_imputed".format(c) for c in ['VWAP', 'Target']],
).setStrategy('mean')

X_train = imputer.fit(X_train).transform(X_train)

Py4JJavaError: An error occurred while calling o295.fit.
: org.apache.spark.SparkException: surrogate cannot be computed. All the values in VWAP are Null, Nan or missingValue(NaN)
	at org.apache.spark.ml.feature.Imputer.fit(Imputer.scala:198)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at java.base/jdk.internal.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.base/java.lang.reflect.Method.invoke(Method.java:566)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:357)
	at py4j.Gateway.invoke(Gateway.java:282)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.ClientServerConnection.waitForCommands(ClientServerConnection.java:182)
	at py4j.C

In [6]:
col_features = ['Count', "Open", "High", "Low", "Close", "Volume", "VWAP", "Target"]

In [7]:
from pyspark.ml.feature import VectorAssembler

vecAssembler = VectorAssembler(inputCols=col_features, outputCol="Independent Features")
output = vecAssembler.transform(X_train)

In [8]:
# Final data
final_data = output.select(["Independent Features", "Target"])
final_data.head(3)

[Row(Independent Features=DenseVector([40.0, 2376.58, 2399.5, 2357.14, 2374.59, 19.233, 2373.1164, -0.0042]), Target=-0.004218152387429286),
 Row(Independent Features=DenseVector([5.0, 8.53, 8.53, 8.53, 8.53, 78.38, 8.53, -0.0144]), Target=-0.014398966468964769),
 Row(Independent Features=DenseVector([229.0, 13835.194, 14013.8, 13666.11, 13850.176, 31.5501, 13827.0621, -0.0146]), Target=-0.014643224355736173)]

In [9]:
# Base Model (Without Time series considerations)
from pyspark.ml.regression import LinearRegression
train_data, test_data = final_data.randomSplit([0.75, 0.25]) # Randomly split the data into training and testing

lr = LinearRegression(maxIter=10, featuresCol="Independent Features", labelCol = "Target")

lrModel = lr.fit(train_data) # Fit the model

Py4JJavaError: An error occurred while calling o137.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 3 in stage 9.0 failed 1 times, most recent failure: Lost task 3.0 in stage 9.0 (TID 53) (192.168.1.3 executor driver): org.apache.spark.SparkException: Failed to execute user defined function (VectorAssembler$$Lambda$3417/0x000000084135d840: (struct<Count:double,Open:double,High:double,Low:double,Close:double,Volume:double,VWAP:double,Target:double>) => struct<type:tinyint,size:int,indices:array<int>,values:array<double>>)
	at org.apache.spark.sql.errors.QueryExecutionErrors$.failedExecuteUserDefinedFunctionError(QueryExecutionErrors.scala:136)
	at org.apache.spark.sql.errors.QueryExecutionErrors.failedExecuteUserDefinedFunctionError(QueryExecutionErrors.scala)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.sort_addToSorter_0$(Unknown Source)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$Generat